# Elliptic Curve Menezes-Qu-Vanstone (ECMQV)

Key exchange

In [1]:
import random
import math
from typing import Tuple

# Addition Law

In [2]:
# y^2 = x^3 + a*x + b

In [3]:
def add_points(P, Q, p):
    x1, y1 = P
    x2, y2 = Q
    
    if x1 == x2 and y1 == y2:
        beta = (3*x1*x2 + a) * pow(2*y1, -1, p)
    else:
        beta = (y2 - y1) * pow(x2 - x1, -1, p)
    
    x3 = (beta*beta - x1 - x2) % p
    y3 = (beta * (x1 - x3) - y1) % p
    
    is_on_curve((x3, y3), p)
        
    return x3, y3

def is_on_curve(P, p):
    x, y = P
    assert (y*y) % p == ( pow(x, 3, p) + a*x + b ) % p
    
def apply_double_and_add_method(G, k, p):
    target_point = G
    
    k_binary = bin(k)[2:] #0b1111111001
    
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]
        
        # doubling - always
        target_point = add_points(target_point, target_point, p)
        
        if current_bit == "1":
            target_point = add_points(target_point, G, p)
    
    is_on_curve(target_point, p)
    
    return target_point

# Curve Configuration

In [4]:
# Secp256k1

# curve formula
a = 0; b = 7

# base point
G = (55066263022277343669578718895168534326250603453777594175500187360389116729240, 
     32670510020758816978083085130507043184471273380659243275938904335757337482424)

# modulo
p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)

# order
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

# cofactor
h = 1

In [5]:
is_on_curve(G, p)

In [6]:
is_on_curve(P = apply_double_and_add_method(G = G, k = n - 1, p = p), p = p)

# ECMQV

In [7]:
def bar(P: Tuple) -> int:
    l = math.ceil( (math.floor(math.log(n, 2)) + 1) / 2 )
    x, y = P
    P_bar = ( x % pow(2, l) ) + pow(2, l)
    return P_bar

# Private - Public Key Pairs

In [8]:
# private key of Alice
ka = random.getrandbits(256)

# public key of Alice
Qa = apply_double_and_add_method(G = G, k = ka, p = p)

# random key of Alice (secret)
ra = random.getrandbits(256)

# random point of Alice (public)
Ra = apply_double_and_add_method(G = G, k = ra, p = p)

In [9]:
# private key of Bob
kb = random.getrandbits(256)

# public key of Bob
Qb = apply_double_and_add_method(G = G, k = kb, p = p)

# random key of Bob (secret)
rb = random.getrandbits(256)

# random point of Bob (public)
Rb = apply_double_and_add_method(G = G, k = rb, p = p)

# Signatures

In [10]:
# Alice sends sa to Bob
sa = ( ra + bar(Ra) * ka ) % n

In [11]:
# Bob sends sb to Alice
sb = ( rb + bar(Rb) * kb ) % n

# Key Exchange

In [12]:
# Alice calculate Ja
# Ja = h x sa x (Rb + Rb_bar x Qb)
Ja = apply_double_and_add_method(G = Qb, k = bar(Rb), p = p)
Ja = add_points(P = Rb, Q = Ja, p = p)
Ja = apply_double_and_add_method(G = Ja, k = h*sa, p = p)

In [15]:
Ja

(91331604342144154413973906479535045783690647037523080316731432691872424541979,
 63523924259333960066627705029355839858064013109919954201524063163418993487432)

In [13]:
# Bob calculates Jb
# Jb = h x sb x (Ra + Ra_bar x Qa)
Jb = apply_double_and_add_method(G = Qa, k = bar(Ra), p = p)
Jb = add_points(P = Ra, Q = Jb, p = p)
Jb = apply_double_and_add_method(G = Jb, k = h*sb, p = p)

In [16]:
Jb

(91331604342144154413973906479535045783690647037523080316731432691872424541979,
 63523924259333960066627705029355839858064013109919954201524063163418993487432)

In [14]:
assert Ja == Jb

# Proof

Alice calculates:

    Ja = h x sa x (Rb + Rb_bar x Qb)

    Ja = h x sa x (rb x G + Rb_bar x kb x G)

    Ja = h x sa x (rb + Rb_bar x kb) x G

    Ja = h x sa x sb x G

Bob calculates: 

    Jb = h x sb x (Ra + Ra_bar x Qa)

    Jb = h x sb x (ra x G + Ra_bar x ka x G)

    Jb = h x sb x (ra + Ra_bar x ka) x G

    Jb = h x sb x sa x G

So, Alice and Bob must calculate same point!

    Ja = Jb